In [ ]:
%pip install langchain neo4j sentence-transformers pandas numpy

In [3]:
# Import necessary libraries 
import pandas as pd
import numpy as np

import langchain 
from langchain_ollama import ChatOllama

from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationSummaryMemory

from neo4j import GraphDatabase

from neo4j.exceptions import ServiceUnavailable
import logging

In [4]:
# Define the Ollama LLM with configurable parameters

ollama_llm = ChatOllama(
    base_url="http://localhost:11434",  # URL of your local Ollama server
    model="llama3.2:latest",            # Model name
    temperature=0.7,                    # Configurable temperature
    num_predict=200                      # Configurable maximum tokens for response
)

In [9]:
# Data preparation

# read the CSV file
data = pd.read_csv('test-data-2.csv')

# Replace NaN values with empty strings for Neo4j compatibility
data = data.replace({np.nan: ''})

# Connect to Neo4j database
driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "test_password"))

# Function to store clothing items in Neo4j
def create_clothing_graph(tx, row):
    query = (
        """
        MERGE (c:Clothing {id: $id})
        SET c += {name: $name, brand: $brand, type: $type, group: $group, details: $details, 
                price: $price, currency: $currency, color: $color, size: $size, 
                style: $style, pattern: $pattern, material: $material, occasion: $occasion}
        MERGE (b:Brand {name: $brand})
        MERGE (t:Type {name: $type})
        MERGE (b)-[:MAKES]->(c)
        MERGE (c)-[:BELONGS_TO]->(t)
        """
    )
    tx.run(query, id=row['id'], name=row['name'], brand=row['brand'], type=row['type'], group=row['group'], 
           details=row['details'], price=row['price'], currency=row['currency'], color=row['color'], 
           size=row['size'], style=row['style'], pattern=row['pattern'], material=row['material'], 
           occasion=row['occasion'])

# Apply the clothing data to the database
def import_clothing_data(data):
    with driver.session() as session:
        data.apply(lambda row: session.execute_write(create_clothing_graph, row), axis=1)

# delete index
def delete_fulltext_index():
    with driver.session() as session:
        session.run("DROP INDEX clothingIndex IF EXISTS")

# create full text index
def create_fulltext_index():
    with driver.session() as session:
        session.run("""
        CREATE FULLTEXT INDEX clothingIndex
        FOR (c:Clothing)
        ON EACH [c.id, c.name, c.brand, c.details]
        """)

# Calling the Delete Index and Create Index functions after a clothing node has been created
delete_fulltext_index()  # Delete the index first (if it exists)
create_fulltext_index()  # Then create a new index

In [17]:
import re

# Full-Text Search in Neo4j and Response Generation

def escape_special_characters(query: str) -> str:
    # Escape characters that are special in Lucene query syntax
    lucene_special_characters = r'([\+\-\!\(\)\{\}\[\]\^\"\~\*\?\:\\\/])'
    return re.sub(lucene_special_characters, r'\\\1', query)

def retrieve_products_fulltext(query):

    # Ensure the query is a valid string
    # Escape special characters in the query
    query_string = escape_special_characters(query)

    # Retrieve products with error handling
    try:
        with driver.session() as session:
            result = session.run("""
            CALL db.index.fulltext.queryNodes('clothingIndex', $query)
            YIELD node, score
            RETURN node.id as id, node.name AS name, node.brand AS brand, node.details AS details, score
            ORDER BY score DESC LIMIT 10
            """, {"query": query_string})
            
            products = []
            for record in result:
                products.append({
                    "id": record["id"], 
                    "name": record["name"],
                    "brand": record["brand"],
                    "details": record["details"],
                    "score": record["score"]
                })
        return products
    except Exception as e:
        logging.error(f"Neo4j Service unavailable: {e}")
        return []

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain
from langchain.chains import LLMChain

# Define prompt template with history, input_query, and product_data
template = """
    You are an AI assistant helping users find clothing.
    Here is the conversation history so far:
    {history}
    The user's current query is: "{input_query}".
    Here are the product details:
    {product_data}.
    Generate a helpful response to suggest relevant products.
"""

prompt = PromptTemplate(input_variables=["history", "input_query", "product_data"], template=template)

# Create summary memory instead of buffer memory
memory = ConversationSummaryMemory(llm=ollama_llm)

# Create a conversation chain using the prompt and memory
conversation_chain = ConversationChain(
    llm=ollama_llm, 
    prompt=prompt, 
    memory=memory
)

# # Create an LLMChain with the prompt
# llm_chain = LLMChain(
#     llm=ollama_llm,
#     prompt=prompt,
#     memory=memory
# )


In [23]:
# Function to search for products and generate a response

def search_and_generate_response(user_query):
    
    logging.info(f"Received user query: {user_query}")

    # Retrieve products from Neo4j
    similar_products = retrieve_products_fulltext(user_query)
    
    if not similar_products:
        return {"response": "No matching products found", "products": []}

    # Format product data for LLM input
    product_data = ", ".join([f"{p['name']} (ID: {p['id']}, Brand: {p['brand']}, Details: {p['details']})" for p in similar_products])
    
    try:
        # Generate a response using the conversation chain
        response = conversation_chain.predict(
            input_query=user_query, 
            product_data=product_data
        )

        
    except Exception as e:
        logging.error(f"LLM response generation failed: {e}")
        response = "There was an issue generating the response. Please try again."

    return {
        "response": response,
        "products": similar_products
    }


In [34]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryMemory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Define prompt template with history, input_query, and product_data
template = """
    You are an AI assistant helping users find clothing.
    Here is the conversation history so far:
    {history}
    The user's current query is: "{input_query}".
    Here are the product details:
    {product_data}.
    Generate a helpful response to suggest relevant products.
"""

prompt = PromptTemplate(input_variables=["history", "input_query", "product_data"], template=template)


# Initialize memory to store conversation history
memory = ConversationSummaryMemory(llm=ollama_llm)

# Function to handle session history
def get_session_history():
    return memory.load_memory_variables({})["history"]

# Create a RunnableWithMessageHistory
conversation_chain = RunnableWithMessageHistory(
    runnable=ollama_llm,
    get_session_history=get_session_history
)

def search_and_generate_response(user_query):
    logging.info(f"Received user query: {user_query}")

    # Retrieve products from Neo4j
    similar_products = retrieve_products_fulltext(user_query)

    if not similar_products:
        return {"response": "No matching products found", "products": []}

    # Format product data for LLM input
    product_data = ", ".join([f"{p['name']} (ID: {p['id']}, Brand: {p['brand']}, Details: {p['details']})" for p in similar_products])

    try:
        # Generate a response using the new chain
        response = conversation_chain.invoke({
            "input_query": user_query,
            "product_data": product_data
        })

    except Exception as e:
        logging.error(f"LLM response generation failed: {e}")
        response = "There was an issue generating the response. Please try again."

    return {
        "response": response,
        "products": similar_products
    }


In [37]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryMemory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import BaseMessage, HumanMessage, AIMessage

# Define prompt template with history, input_query, and product_data
template = """
    You are an AI assistant helping users find clothing.
    Here is the conversation history so far:
    {history}
    The user's current query is: "{input_query}".
    Here are the product details:
    {product_data}.
    Generate a helpful response to suggest relevant products.
"""

prompt = PromptTemplate(input_variables=["history", "input_query", "product_data"], template=template)

# Initialize memory to store conversation history
memory = ConversationSummaryMemory(llm=ollama_llm)

# Function to handle session history, converting it to message objects
def get_session_history():
    # Load the memory variables which returns history as a string
    conversation_history = memory.load_memory_variables({})["history"]

    # Convert the string history into a list of message objects for the LLM
    messages = []
    if conversation_history:
        history_lines = conversation_history.split("\n")
        for line in history_lines:
            if "User:" in line:
                messages.append(HumanMessage(content=line.replace("User:", "").strip()))
            elif "AI:" in line:
                messages.append(AIMessage(content=line.replace("AI:", "").strip()))
    
    return messages

# Create a RunnableWithMessageHistory
conversation_chain = RunnableWithMessageHistory(
    runnable=ollama_llm,
    get_session_history=get_session_history
)

def search_and_generate_response(user_query):
    logging.info(f"Received user query: {user_query}")

    # Retrieve products from Neo4j
    similar_products = retrieve_products_fulltext(user_query)

    if not similar_products:
        return {"response": "No matching products found", "products": []}

    # Format product data for LLM input
    product_data = ", ".join([f"{p['name']} (ID: {p['id']}, Brand: {p['brand']}, Details: {p['details']})" for p in similar_products])

    try:
        # Generate a response using the new chain
        response = conversation_chain.invoke({
            "input_query": user_query,
            "product_data": product_data
        })

    except Exception as e:
        logging.error(f"LLM response generation failed: {e}")
        response = "There was an issue generating the response. Please try again."

    return {
        "response": response,
        "products": similar_products
    }


In [41]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryMemory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import BaseMessage, HumanMessage, AIMessage

# Define prompt template with history, input_query, and product_data
template = """
    You are an AI assistant helping users find clothing.
    Here is the conversation history so far:
    {history}
    The user's current query is: "{input_query}".
    Here are the product details:
    {product_data}.
    Generate a helpful response to suggest relevant products.
"""

prompt = PromptTemplate(input_variables=["history", "input_query", "product_data"], template=template)

# Initialize memory to store conversation history
memory = ConversationSummaryMemory(llm=ollama_llm)

# Custom wrapper class to hold the messages with a 'messages' attribute
class MessageHistory:
    def __init__(self, messages):
        self.messages = messages

# Function to handle session history and wrap it in a MessageHistory object
def get_session_history():
    # Load the memory variables which returns history as a string
    conversation_history = memory.load_memory_variables({})["history"]

    # Convert the string history into a list of message objects for the LLM
    messages = []
    if conversation_history:
        history_lines = conversation_history.split("\n")
        for line in history_lines:
            if "User:" in line:
                messages.append(HumanMessage(content=line.replace("User:", "").strip()))
            elif "AI:" in line:
                messages.append(AIMessage(content=line.replace("AI:", "").strip()))
    
    # Wrap the list of messages in a MessageHistory object
    return MessageHistory(messages)

# Create a RunnableWithMessageHistory
conversation_chain = RunnableWithMessageHistory(
    runnable=ollama_llm,
    get_session_history=get_session_history
)

def search_and_generate_response(user_query):
    logging.info(f"Received user query: {user_query}")

    # Retrieve products from Neo4j
    similar_products = retrieve_products_fulltext(user_query)

    if not similar_products:
        return {"response": "No matching products found", "products": []}

    # Format product data for LLM input
    product_data = ", ".join([f"{p['name']} (ID: {p['id']}, Brand: {p['brand']}, Details: {p['details']})" for p in similar_products])

    try:
        # Prepare the input for the conversation chain
        history = get_session_history().messages
        
        # Generate a response using the new chain
        response = conversation_chain.invoke({
            "history": history,  # Ensure history is passed
            "input_query": user_query,
            "product_data": product_data
        })

    except Exception as e:
        logging.error(f"LLM response generation failed: {e}")
        response = "There was an issue generating the response. Please try again."

    return {
        "response": response,
        "products": similar_products
    }


In [ ]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
    You are an AI assistant helping users find clothing.
    The user's current query is: {user_query}.
    Here are the product details:
    {context}.
    Generate a helpful response to suggest relevant products.
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": retrieve_products_fulltext,
            "user_query": RunnablePassthrough(),
        }
    | prompt
    | ollama_llm
    | StrOutputParser()
)

chain.invoke(input={"user_query": "What are some good shoes for running?"})

TypeError: expected string or bytes-like object, got 'set'

In [1]:
# topic detection 
on_topic_keywords = ['jeans', 'dress', 'shirt', 'pants', 'clothing', 'size', 'color', 'fashion', 'outfit', 'trousers', 'jacket']
off_topic_keywords = ['weather', 'news', 'movie', 'sports', 'politics', 'celebrity']

def detect_topic(user_input):
    if any(word in user_input.lower() for word in on_topic_keywords):
        return "on_topic"
    elif any(word in user_input.lower() for word in off_topic_keywords):
        return "off_topic"
    else:
        return "neutral"

# Example usage
user_input = "I'm looking for a white shirt with brand Huili"

if detect_topic(user_input) == "on_topic":
    result = search_and_generate_response(user_input)
    print(result)
elif detect_topic(user_input) == "off_topic":
    print({"response": "It seems your question is off-topic. Do you want to search for clothing?"})


NameError: name 'search_and_generate_response' is not defined